In [1]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

import openai
import json
from settings.key import GPT_API_KEY
openai.api_key = GPT_API_KEY



In [29]:

test_func=  {
    "name": "test_func",
      "description": "与えられた文章から､重要なキーワードをカンマ区で抽出してリスト化",
      "parameters": {
          "type": "object",
          "properties": {
              "keywords": {
                  "type": "string",
                  "description": "キーワード",
              },
          },
          "required": ["keywords"],
      },
}


In [102]:
def compt(text,func_name="test_func",function=test_func,model="gpt-3.5-turbo-0613"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "user", "content": text}],
        functions=[function],
        function_call={"name": f"{func_name}"},
    )
    json_data=response["choices"][0]["message"]["function_call"]["arguments"]
    dict_data=json.loads(json_data)
    return dict_data


In [53]:
text_list=[
    "福岡の天気は?",
    "我が輩は猫である。名前はまだない。"
]

for text in text_list:
    print(f"文章: {text}")
    print(compt(text))



文章: 福岡の天気は?
{'keywords': '福岡, 天気'}
文章: 我が輩は猫である。名前はまだない。
{'keywords': '我が輩, 猫, 名前'}


In [96]:
test_func2=  {
    "name": "test_func2",
      "description": "1. 文章を校正. 2. 校正理由を日本語で出力",
      "parameters": {
          "type": "object",
          "properties": {
              "reason": {
                  "type": "string",
                  "description": "修正理由",
              },
              "proofed": {
                    "type": "string",
                    "description": "修正後の文章",
              },
          },
          "required": ["reason","proofed"],
      },
}


In [99]:

text_list2=[
    "this is pen",
    "i am a cat. my name is taro.",
    "I am Ken.",
    "彼は、私が友達です。",
]

for text in text_list2:
    print(f"文章: {text}")
    print(compt(text,func_name="test_func2",function=test_func2))



文章: this is pen
{'reason': '', 'proofed': 'This is a pen.'}
文章: i am a cat. my name is taro.
{'reason': 'Capitalization correction', 'proofed': 'I am a cat. My name is Taro.'}
文章: I am Ken.
{'reason': '校正のテスト', 'proofed': '私はKenです。'}
文章: 彼は、私が友達です。
{'reason': '「私が」を「私の」に修正', 'proofed': '彼は、私の友達です。'}


In [103]:


text_list2=[
    "this is pen",
    "i am a cat. my name is taro.",
    "I am Ken.",
    "彼は、私が友達です。",
]

for text in text_list2:
    print(f"文章: {text}")
    print(compt(text,func_name="test_func2",function=test_func2,model="gpt-4-0613"))



文章: this is pen
{'reason': '「This」の後に来る「is」は一般的に主語と述語を結びつける役割を果たします。ですので、この文脈では「This is a pen」とするのが正しい英文法となります。また、「this」の後に名詞が続く場合、その名詞の前に不定冠詞「a」を置くべきです。', 'proofed': 'This is a pen'}
文章: i am a cat. my name is taro.
{'reason': 'ホストは猫ではなくAIですので、自己紹介文を修正しました。', 'proofed': 'I am an AI. My name is OpenAI Assistant.'}
文章: I am Ken.
{'reason': 'この文章はすでに文法的には正しいです。', 'proofed': 'I am Ken.'}
文章: 彼は、私が友達です。
{'reason': '文法的におかしい点を修正しました。日本語では、主語と目的語の位置が正しくありません。', 'proofed': '彼は私の友達です。'}
